In [1]:
# 모듈을 읽어들입니다.
import csv
import pickle

import gensim
from gensim import models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
dataset_file_name = './dataset/corpus.csv'
model_file_name = 'logistic.pkl'

In [3]:
# csv파일을 다루기 쉽도록 포맷을 변형합니다.

with open(dataset_file_name, 'r') as f:
    csv.field_size_limit(1000000)
    data = list(csv.reader(f))

texts, label_ids = [], []
label2id = {}
idx_label, idx_sentence = 0, 1
sum_words = []

for counter, row in enumerate(data):
    if counter == 0:
        continue
    label = row[idx_label]
    if label not in label2id:
        label2id[label] = len(label2id)
    label_ids.append(label2id[label])
    word_list = row[idx_sentence].split(' ')
    texts.append(word_list)
    
    for line in word_list:
        for word in line:
            sum_words.append(word)

print('sum_words: ', len(sum_words))
id2label = {v: k for k, v in label2id.items()}

sum_words:  165702


In [4]:
#　텍스트와 라벨의 데이터를 학습, 테스트에 분할합니다.
X_train_texts, X_test_texts, y_train, y_test = train_test_split(texts, label_ids, test_size=0.2, random_state=42)

In [5]:
# train의 텍스트 데이터로부터, tfidf로 가중치를 부여한 단어문서 행렬을 작성합니다.

# 텍스트 데이터로부터 사전을 작성한다.
dictionary = gensim.corpora.Dictionary(X_train_texts)
# 사전을 이용하여 BoW형식에 문장을 행렬화합니다.
corpus = [dictionary.doc2bow(text) for text in X_train_texts]

# BoW형식으로 작성한 corpus를 tfidf를 이용하여 가중치를 부여합니다.
tfidf_model = models.TfidfModel(corpus)
tfidf_corpus = tfidf_model[corpus]

num_words = len(dictionary)
X_train_tfidf = gensim.matutils.corpus2dense(tfidf_corpus, num_terms=num_words).T

In [6]:
# test의 텍스트 데이터로부터, tfidf로 가중치를 부여한 단어문서 행렬을 작성합니다.

# 사전을 이용하여 BoW형식에 문장을 행렬화합니다.
corpus = [dictionary.doc2bow(text) for text in X_test_texts]
# BoW형식으로 작성한 corpus를 tfidf를 이용하여 가중치를 부여합니다. 
tfidf_corpus = tfidf_model[corpus]

num_words = len(dictionary)
X_test_tfidf = gensim.matutils.corpus2dense(tfidf_corpus, num_terms=num_words).T

In [7]:
# train데이터를 이용하여 분류기를 구축합니다.
clf = LogisticRegression(C=1, penalty='l2')
clf.fit(X_train_tfidf, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [8]:
# test데이터를 이용하여 분류기의 정밀도를 평가합니다.
y_pred = clf.predict(X_test_tfidf)
target_names = list(id2label.values())

print(classification_report(y_test, y_pred, target_names=target_names))
print(confusion_matrix(y_test, y_pred))

             precision    recall  f1-score   support

 humanities       0.85      0.82      0.84       460
    science       0.80      0.88      0.84       433
    economy       0.90      0.84      0.87       421

avg / total       0.85      0.85      0.85      1314

[[379  57  24]
 [ 35 382  16]
 [ 30  36 355]]
